[View in Colaboratory](https://colab.research.google.com/github/shubhamitradas/Toxicity-Challenge/blob/master/toxic_pytorch.ipynb)

In [1]:
!pip install http://download.pytorch.org/whl/cpu/torch-0.3.1-cp36-cp36m-linux_x86_64.whl
!pip install torchvision

    100% |████████████████████████████████| 47.0MB 20.7MB/s 
    100% |████████████████████████████████| 61kB 4.0MB/s 
    100% |████████████████████████████████| 2.0MB 13.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import Dataset, DataLoader


from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Parameters and DataLoaders
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_EPOCHS = 3

MAX_SEQUENCE_LENGTH = 150
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1



# Some utility functions
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def create_variable(tensor):
    # Do cuda() before wrapping with variable
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)


    
path = ''
EMBEDDING_FILE=path+'glove.840B.300d.txt'
TRAIN_DATA_FILE=path+'train_toxic.csv'
TEST_DATA_FILE=path+'test_toxic.csv'

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

########################################
## process texts in datasets
########################################
print('Processing text dataset')

#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)

#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)



class TrainTestDataset(Dataset):


    # Initialize your data, download, etc.
    def __init__(self, isTrain=True):
        train_df = pd.read_csv(TRAIN_DATA_FILE,nrows=900) 
        test_df = pd.read_csv(TEST_DATA_FILE)
        print("test size",test_df.shape)
        
        list_sentences_train = train_df["comment_text"].fillna("NA").values
        list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
        labels_train = train_df[list_classes].values
        list_sentences_test = test_df["comment_text"].fillna("NA").values
        
        comments = []
        for text in list_sentences_train:
            comments.append(text_to_wordlist(text))

    
        test_comments=[]
        for text in list_sentences_test:
            test_comments.append(text_to_wordlist(text))


        tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
        tokenizer.fit_on_texts(comments + test_comments)

        sequences = tokenizer.texts_to_sequences(comments)
        test_sequences = tokenizer.texts_to_sequences(test_comments)


        word_index = tokenizer.word_index
        print('Found %s unique tokens' % len(word_index))


        data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
        print('Shape of data tensor:', data.shape)
        print('Shape of label tensor:', labels_train.shape)


        test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
        print('Shape of test_data tensor:', test_data.shape)
        self.train = isTrain
        if(self.train == True):
            self.len = data.shape[0]
        else:
            self.len = test_data.shape[0]
        self.x_data = torch.LongTensor(data)
        self.x_test_data = torch.LongTensor(test_data)
        self.y_data = torch.LongTensor(labels_train)


    def __getitem__(self, index):
        if(self.train == True):
            return self.x_data[index], self.y_data[index]
        else:
            return self.x_test_data[index]
        

    def __len__(self):
        return self.len





Using TensorFlow backend.


Processing text dataset


In [4]:
train_dataset = TrainTestDataset(isTrain=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=256,
                          shuffle=True)


test_dataset = TrainTestDataset(isTrain=False)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=1024,
                          shuffle=True)


test size (153164, 2)
Found 251665 unique tokens
Shape of data tensor: (900, 150)
Shape of label tensor: (900, 6)
Shape of test_data tensor: (153164, 150)
test size (153164, 2)
Found 251665 unique tokens
Shape of data tensor: (900, 150)
Shape of label tensor: (900, 6)
Shape of test_data tensor: (153164, 150)


In [0]:
import torch.nn.functional as F


class RNNClassifier(nn.Module):
    # Our model

    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1
        #print("Input to embedding  is ",input_size,hidden_size)
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional)
        self.dropout = nn.Dropout(0.5)
        self.bn = nn.BatchNorm1d(256)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input):
        # Note: we run this all at once (over the whole input sequence)
        # input shape: B x S (input size)
        # transpose to make S(sequence) x B (batch)
        #print("Inside forward1 ",input.shape)
        input = input.t()
        batch_size = input.size(1)
        
        #print(batch_size)
        

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        # Embedding S x B -> S x B x I (embedding size)
        #print("Inside forward")
        #print(input.shape)

        embedded = self.embedding(input)
        #print(embedded.shape)
        

        # Pack them up nicely
        '''gru_input = pack_padded_sequence(
            embedded, seq_lengths.data.cpu().numpy())'''
        gru_input = embedded
        

        # To compact weights again call flatten_parameters().
        self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)
        hidden = self.dropout(hidden)
        #hidden = self.bn(hidden)

        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden[-1])
        #print("fc_output",fc_output)
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size)
        return create_variable(hidden)




def train():
    
        total_loss = 0

        for i, data in enumerate(train_loader, 0):
           # get the inputs
           inputs, labels = data
           # wrap them in Variable
           inputs, labels = create_variable(inputs), create_variable(labels)
           output = classifier(inputs)
           pred = F.softmax(output)
           loss = criterion(pred, labels)
           #print("loss: ",loss.data)
           total_loss += loss.data[0]
           classifier.zero_grad()
           loss.backward()
           optimizer.step()
        
        print("total_loss",total_loss)    
        return total_loss




# Testing cycle
def test(name=None):
        
        output = classifier(data_val)
        print("In test :",output.shape)
        pred = output #.data.max(1, keepdim=True)[1]
        print(pred)
        print(labels_val)
        '''correct = pred.eq(labels_val.data.view_as(pred)).cpu().sum()

        print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))'''
        from sklearn import metrics
        #roc = metrics.roc_auc_score(labels_val.data.cpu().numpy(), pred.data.cpu().numpy())
        #print("ROC:", roc)




In [13]:
classifier = RNNClassifier(MAX_NB_WORDS , HIDDEN_SIZE, 6, N_LAYERS)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
    classifier = nn.DataParallel(classifier)

if torch.cuda.is_available():
    classifier.cuda()

optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.MultiLabelMarginLoss()

start = time.time()
print("Training for %d epochs..." % N_EPOCHS)
for epoch in range(1, 10 + 1):
    # Train cycle
    train()
    

Training for 3 epochs...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


total_loss 15.110389947891235
total_loss 4.805549144744873
total_loss 2.5375065207481384
total_loss 1.8945719599723816
total_loss 1.2717264294624329
total_loss 0.8551862984895706
total_loss 0.7458672672510147
total_loss 0.6911358386278152
total_loss 0.6703790575265884
total_loss 0.6903959214687347


In [7]:
import torch.nn.functional as F
predictions = []
for i, test_data in enumerate(test_loader, 0):
    #print(test_data.shape)
    test_data = create_variable(test_data)
    output = classifier(test_data)
    pred = F.sigmoid(output)
    predictions.append(pred.data.cpu().numpy())

    
predictions = np.vstack(predictions)
print("===> Raw predictions done. Here is a snippet")
print(predictions)

===> Raw predictions done. Here is a snippet
[[0.9699192  0.8078989  0.08396397 0.41501966 0.16176644 0.10449158]
 [0.98990685 0.80779487 0.08335509 0.25004372 0.15071873 0.03490622]
 [0.9734914  0.6271875  0.12660392 0.18886292 0.26125896 0.25229114]
 ...
 [0.9645737  0.5643977  0.08189034 0.24476935 0.19861893 0.19117911]
 [0.95705944 0.65933484 0.11777349 0.14895216 0.1911027  0.15711537]
 [0.9848856  0.7690941  0.06945001 0.2642095  0.13213487 0.36221263]]


In [8]:
print(predictions.shape)
sample_submission = pd.read_csv("sample_toxic.csv")
print(sample_submission.shape)
sample_submission[list_classes] = predictions

sample_submission.to_csv('submission_pytorch.csv', index=False)

(153164, 6)
(153164, 7)


In [0]:
!gzip submission_pytorch.csv

In [0]:
from google.colab import files


files.download('submission_pytorch.csv.gz')